In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('../data/train_data_fix_concat.csv')
df_test = pd.read_csv('../data/test_data_fix_concat.csv')
df.head(3)

,oil_type,blend_id,oil_property_param_title,oil_property_param_value,component_name,component_class,polymer,component_property_param_title,component_property_param_value,smiles
0,3fa07e0a-415c-496d-b88b-557855cb3e77,49743a76-a614-11ee-9529-005056921581,4c7a51f1-dc82-41dc-92fa-772535c2c70c,15.58,615537f6-1f8f-4240-a5e9-8f7be344ecd3,базовое масло 1 гр,no,02236ee6-5eec-4368-a2e4-6f2e73fb0f96,0.0000,CCCCC
1,3fa07e0a-415c-496d-b88b-557855cb3e77,49743a76-a614-11ee-9529-005056921581,4c7a51f1-dc82-41dc-92fa-772535c2c70c,15.58,615537f6-1f8f-4240-a5e9-8f7be344ecd3,базовое масло 1 гр,no,2511714c-ab50-4566-bc92-8e4095d87d01,0.0000,CCCCC
2,3fa07e0a-415c-496d-b88b-557855cb3e77,49743a76-a614-11ee-9529-005056921581,4c7a51f1-dc82-41dc-92fa-772535c2c70c,15.58,615537f6-1f8f-4240-a5e9-8f7be344ecd3,базовое масло 1 гр,no,38b06d9a-bcf2-40de-8e1b-9c3988c42406,0.0001,CCCCC


Заполним пропуски в данных тупыми значениями

In [7]:
df['oil_type'] = df['oil_type'].fillna(value='unknown')
df['oil_property_param_value'] = df['oil_property_param_value'].fillna(value=-1)
df['component_class'] = df['component_class'].fillna(value='unknown')
df['component_property_param_value'] = df['component_property_param_value'].fillna(value=-1)
df['smiles'] = df['smiles'].fillna(value='unknown')
df['polymer'] = df['polymer'].fillna(value='unknown')

И в тесте тоже

In [8]:
df_test['oil_type'] = df_test['oil_type'].fillna(value='unknown')
df_test['oil_property_param_value'] = df_test['oil_property_param_value'].fillna(value=-1)
df_test['component_class'] = df_test['component_class'].fillna(value='unknown')
df_test['component_property_param_value'] = df_test['component_property_param_value'].fillna(value=-1)
df_test['smiles'] = df_test['smiles'].fillna(value='unknown')
df_test['polymer'] = df_test['polymer'].fillna(value='unknown')

Сделаем список из blend_id в train датасете у которых в свойствах есть наш таргет-свойство

In [9]:
blend_ids_with_target = df[df['oil_property_param_title'] == 'ad7e6027-00b8-4c27-918c-d1561f949ad8']['blend_id'].unique().tolist()

In [10]:
df = df[df['blend_id'].isin(blend_ids_with_target)].reset_index(drop=True)

Создаем таблицу с колонками component_property

In [11]:
pivot_df = df.pivot_table(index=['oil_type', 'blend_id', 'oil_property_param_title', 'oil_property_param_value', 'component_name', 'component_class', 'smiles', 'polymer'],
                          columns='component_property_param_title',
                          values='component_property_param_value',
                          fill_value=np.nan).reset_index()

# Rename columns
pivot_df.columns.name = None

Также в тесте

In [12]:
pivot_df_test = df_test.pivot_table(index=['oil_type', 'blend_id', 'oil_property_param_title', 'oil_property_param_value', 'component_name', 'component_class', 'smiles', 'polymer'],
                          columns='component_property_param_title',
                          values='component_property_param_value',
                          fill_value=np.nan).reset_index()

# Rename columns
pivot_df_test.columns.name = None

In [13]:
print(f'Размер датасета train: {pivot_df.shape}')
print(f'Размер датасета test: {pivot_df_test.shape}')

Размер датасета train: (16797, 51)
Размер датасета test: (6420, 55)


В данный момент в наших смесях из трейн датасета здесь есть свойство-таргет, значение которого нужно занести в новую колонку

Посмотрим на значения этого свойства для каждого бленда

In [14]:
pivot_df[pivot_df['oil_property_param_title'] == 'ad7e6027-00b8-4c27-918c-d1561f949ad8'][['blend_id', 'oil_property_param_value']].drop_duplicates().head(2)

,blend_id,oil_property_param_value
42,007b4d7a-b1f8-11ed-93b4-005056921581,13360.0
70,04aed446-6beb-11ed-87f9-005056921581,7416.0


In [15]:
index_columns = pivot_df.drop(columns=['oil_property_param_title', 'oil_property_param_value']).columns

In [16]:
targets_values = pivot_df[pivot_df['oil_property_param_title'] == 'ad7e6027-00b8-4c27-918c-d1561f949ad8'][['blend_id', 'oil_property_param_value']].drop_duplicates()

In [17]:
merged_df = pd.merge(pivot_df, targets_values, on='blend_id', how='left')

In [110]:
merged_df = merged_df[merged_df['oil_property_param_title'] != 'ad7e6027-00b8-4c27-918c-d1561f949ad8']

In [111]:
merged_df = merged_df.rename(columns={'oil_property_param_value_y': 'target', 'oil_property_param_value_x': 'oil_property_param_value'})

In [112]:
merged_df.shape

(14716, 51)

Добавим колонку target заполненную nan к pivot_df_test

In [113]:
pivot_df_test['target'] = np.nan

In [114]:
final_df = pd.concat([merged_df, pivot_df_test], ignore_index=True)

In [126]:
final_df.shape

(21136, 84)

In [130]:
final_df.columns

Index(['oil_type', 'blend_id', 'oil_property_param_title',
       'oil_property_param_value', 'component_name', 'component_class',
       'smiles', 'component_property_009c5608-34ee-44ec-a294-a6252d691af3',
       'component_property_02236ee6-5eec-4368-a2e4-6f2e73fb0f96',
       'component_property_0817d2c9-2478-40d2-aab3-6b055f8071f0',
       'component_property_08db8d40-6b81-44ab-ab91-4b482ebc4187',
       'component_property_0bede6ba-5118-4a24-8c61-4df27f5746d7',
       'component_property_0ec6a170-4283-4466-8e16-ab71675053d6',
       'component_property_1931ea10-9783-4065-a77c-04532d7d0559',
       'component_property_1a8ce77b-9eac-489d-b6fc-4d7df0b74092',
       'component_property_1da38bd4-1cca-467b-b480-76c4fac65d5f',
       'component_property_2511714c-ab50-4566-bc92-8e4095d87d01',
       'component_property_266a1a34-14d1-49f7-b1ca-d354f9ff6ce8',
       'component_property_2a6e6683-e8f2-49ae-8114-b752a1e97c60',
       'component_property_2caf7dfa-de92-43ba-811d-ce9bbac9443a',
 

In [127]:
index_columns = final_df.drop(columns=['oil_property_param_title', 'oil_property_param_value']).columns.to_list()

In [128]:
final_df_pivot = final_df.pivot_table(index=index_columns,
                          columns='oil_property_param_title',
                          values='oil_property_param_value',
                          fill_value=np.nan).reset_index()

# Rename columns
final_df_pivot.columns.name = None

In [129]:
final_df_pivot

,oil_type,blend_id,component_name,component_class,smiles,component_property_009c5608-34ee-44ec-a294-a6252d691af3,component_property_02236ee6-5eec-4368-a2e4-6f2e73fb0f96,component_property_0817d2c9-2478-40d2-aab3-6b055f8071f0,component_property_08db8d40-6b81-44ab-ab91-4b482ebc4187,component_property_0bede6ba-5118-4a24-8c61-4df27f5746d7,...,component_property_94d89046-80c9-41a8-9d0c-d587d9ed18c2,component_property_9afb4d3a-4889-4144-a2fe-647fc146776b,component_property_9e2de643-ddca-487e-b9e7-69b25d8662bf,component_property_a21ee2d5-8f50-40ce-b0b4-d6aec1130eee,component_property_a8bc9c11-42cb-4452-8cc8-8f96eb3d7705,component_property_b38cd6d1-cc71-4771-9d9c-7b37ae274fe9,component_property_c8ae5de6-e2ae-45ed-bf9a-53e0fba25b52,component_property_d6eff30f-788e-48a2-b2bd-cbba188f4e63,component_property_ebe9475f-b29f-4ffd-86af-a16a8321f1ee,component_property_f6cc205c-a44d-40e5-9462-2eab4a673e2a


In [22]:
# train_df = final_df[~pd.isna(final_df['target'])]
# test_df = final_df[pd.isna(final_df['target'])] 

In [23]:
# train_df.shape

In [24]:
# train_df.to_csv('data/train_preprocessed.csv', index=False)
# test_df.to_csv('data/test_preprocessed.csv', index=False)